<a href="https://colab.research.google.com/github/JayCh0711/Data_Science_Projects/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Collection and Preprocessing

In [ ]:
fake_news_dataset = pd.read_csv('/content/_train.csv')

In [ ]:
fake_news_dataset.shape

(16646, 1)

In [ ]:
fake_news_dataset.head()

,text;label
0,dark agenda behind globalism open border altma...
1,america poor still get shaft sami jamil jadall...
2,number accuser grow former miss finland accuse...
3,heroic prego advertisement replaces refresh we...
4,russia syria debbie reynolds thursday even bri...


In [ ]:
#counting the missing value
fake_news_dataset.isnull().sum()

,0
text;label,0


In [ ]:
#replacing null values with empty string | there are no missing values but for practice i am doing this
fake_news_dataset = fake_news_dataset.fillna('')

In [ ]:
#Merging the columns
fake_news_dataset['content'] = fake_news_dataset['text;label']
print(fake_news_dataset['content'])

0        dark agenda behind globalism open border altma...
1        america poor still get shaft sami jamil jadall...
2        number accuser grow former miss finland accuse...
3        heroic prego advertisement replaces refresh we...
4        russia syria debbie reynolds thursday even bri...
                               ...                        
16641    comment cost selfdriving taxi really sixtyseve...
16642    interest duterte get billion china get money j...
16643    forget encyclopaedia get pokiespedia home win ...
16644    u election race huma abedin connection minute ...
16645    shameful obama legacy white man beat viciously...
Name: content, Length: 16646, dtype: object


In [ ]:
#Separating the data and label
X = fake_news_dataset.drop(columns='text;label', axis=1)
Y = fake_news_dataset['text;label']

In [ ]:
print(X)
print(Y)

                                                 content
0      dark agenda behind globalism open border altma...
1      america poor still get shaft sami jamil jadall...
2      number accuser grow former miss finland accuse...
3      heroic prego advertisement replaces refresh we...
4      russia syria debbie reynolds thursday even bri...
...                                                  ...
16641  comment cost selfdriving taxi really sixtyseve...
16642  interest duterte get billion china get money j...
16643  forget encyclopaedia get pokiespedia home win ...
16644  u election race huma abedin connection minute ...
16645  shameful obama legacy white man beat viciously...

[16646 rows x 1 columns]
0        dark agenda behind globalism open border altma...
1        america poor still get shaft sami jamil jadall...
2        number accuser grow former miss finland accuse...
3        heroic prego advertisement replaces refresh we...
4        russia syria debbie reynolds thursday even br

Stemming :
Process of reducing a word to it's root word

Examples; actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [ ]:
fake_news_dataset['content'] = fake_news_dataset['content'].apply(stemming)

In [ ]:
print(fake_news_dataset['content'])

0        dark agenda behind global open border altmarke...
1        america poor still get shaft sami jamil jadall...
2        number accus grow former miss finland accus tr...
3        heroic prego advertis replac refresh webpag pr...
4        russia syria debbi reynold thursday even brief...
                               ...                        
16641    comment cost selfdriv taxi realli sixtyseven c...
16642    interest dutert get billion china get money ja...
16643    forget encyclopaedia get pokiespedia home win ...
16644    u elect race huma abedin connect minut video n...
16645    shame obama legaci white man beat vicious vote...
Name: content, Length: 16646, dtype: object


In [ ]:
#Separating the data and label
X = fake_news_dataset['content'].values
Y = fake_news_dataset['text;label'].values

In [ ]:
print(X)
print(Y)

['dark agenda behind global open border altmarket brandon smith peopl unfamiliar liberti movement stumbl onto undeni fact conspiraci global tend look easi answer understand exist peopl today condit perceiv event misinterpret standpoint occam razor wrongli assum simpl explan probabl right one fact occam razor state instead summar state simpl explan give evid hand probabl right explan well know document decad push global deliber focu effort part select elit intern financi central banker polit leader numer member exclus think tank often openli admit goal total global public perhap believ uneduc common would never read anyway carrol quigley mentor bill clinton member council foreign relat often quot open admiss gener scheme power financi capit farreach aim noth less creat world system financi control privat hand abl domin polit system countri economi world whole system control feudalist fashion central bank world act concert secret agreement arriv frequent meet confer apex system bank inte

In [ ]:
#Converting the textual data to numrerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 226)	0.032306192480828624
  (0, 234)	0.015918788439067556
  (0, 248)	0.025630116885556126
  (0, 482)	0.038908705552746774
  (0, 547)	0.016518029818277893
  (0, 623)	0.012195135155913738
  (0, 631)	0.022941044249407194
  (0, 640)	0.012292213049043273
  (0, 652)	0.013138891819509065
  (0, 682)	0.023953485377687676
  (0, 866)	0.01155663641947367
  (0, 875)	0.02242123772457062
  (0, 877)	0.015930146974593357
  (0, 904)	0.01863798238887868
  (0, 954)	0.015494129259902246
  (0, 1058)	0.016763709022976223
  (0, 1074)	0.017364213697447693
  (0, 1216)	0.014672325822467644
  (0, 1232)	0.01659690939642273
  (0, 1246)	0.016297910452905746
  (0, 1346)	0.016025978524081413
  (0, 1486)	0.028339118499888214
  (0, 1489)	0.01578476076741514
  (0, 1509)	0.013948194082209622
  (0, 2417)	0.011177505965428792
  :	:
  (16645, 69132)	0.06090231710165394
  (16645, 69169)	0.07945748170181238
  (16645, 75700)	0.03474633212403185
  (16645, 76469)	0.046932086272056334
  (16645, 76843)	0.2685053000704022
  (1

Splitting the dataset to training and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=1)

Training a Model : Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

Evaluation

In [ ]:
#Accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
#Accuracy score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score on training data : ', training_data_accuracy)
print('Accuracy score on test data : ', test_data_accuracy)

Making a Predictive System

In [ ]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("The news is real")
else:
  print("The news is fake")

In [ ]:
print(Y_test[3])